In [1]:
from utils import *

from tqdm import tqdm
from pprint import pprint
import json
import _pickle as pkl
from time import time

In [2]:
glove = setup_glove()
print(glove.vectors.size())
VOCAB_SIZE = glove.vectors.size()[0]
with open('../data/data.json', 'r') as f:
    data = json.load(f)

torch.Size([400004, 50])


In [3]:
idx = 5
example_X = (data['X_train'][idx])
example_y = (data['y_train'][idx])
print("ID:", example_X[0])
print("Context:", example_X[1])
print("Question:", example_X[2])
print("Answer Span:", example_y)
print("Answer:", example_X[3])

X, y = make_data([example_X], [example_y], 1)
pprint(y)
print(get_answer_span(y[0], example_X[1]))
num_ex_train = 128
num_ex_val = 32
X_train, y_train = make_data(data['X_train'], data['y_train'], num_ex_train)
X_val, y_val = make_data(data['X_val'], data['y_val'], num_ex_val)
print(len(X_train), len(y_train), len(X_val), len(y_val))

ID: 56ce602faab44d1400b88713
Context: A solar chimney (or thermal chimney, in this context) is a passive solar ventilation system composed of a vertical shaft connecting the interior and exterior of a building. As the chimney warms, the air inside is heated causing an updraft that pulls air through the building. Performance can be improved by using glazing and thermal mass materials in a way that mimics greenhouses.
Question: What kind of system is a solar chimney?
Answer Span: [59, 84]
Answer: passive solar ventilation
[(11, 13)]
passive solar ventilation
Skipped: 2
128 128 32 32


In [4]:
from models import *

conf = {"vocab": glove.vectors,
        "learning_rate": 0.1,
        "epochs": 10,
        "hidden_size": 32,
        "batch_size": 32,
        "opt": "Adam",
        "n_layers": 1}
model = ModelV2(conf)
print(model)
model_name = "%s_D%s_B%s_E%s_H%s_LR%s"%(type(model).__name__, num_ex_train, model.batch_size, model.epochs, model.hidden_size, model.lr)
print(model_name)

ModelV2(
  (encoder_c): Embedding(400004, 50)
  (encoder_q): Embedding(400004, 50)
  (gru_c): GRU(50, 32)
  (gru_q): GRU(50, 32)
  (lin_q): Linear(in_features=32, out_features=32, bias=True)
  (gru_coatt): GRU(96, 128, bidirectional=True)
  (gru_bmod): GRU(256, 32)
  (ans_ptr_1): Linear(in_features=256, out_features=32, bias=True)
  (ans_ptr_2): Linear(in_features=32, out_features=32, bias=True)
  (ans_ptr_3): Linear(in_features=32, out_features=1, bias=True)
  (decoder_start): Linear(in_features=128, out_features=600, bias=True)
  (decoder_end): Linear(in_features=128, out_features=600, bias=True)
)
ModelV2_D128_B32_E10_H32_LR0.1


In [5]:
tic = time()
v_preds, losses, vlosses = model.fit((X_train, y_train), (X_val, y_val))
toc = time()
print("took", toc-tic, "seconds")
torch.save(model, '../evaluation/models/%s'%model_name)
import matplotlib.pyplot as plt
plt.plot(list(range(len(losses))), losses)
plt.plot(list(range(len(vlosses))), vlosses)
plt.show()

batch_size: 32
batches: 4.0
epoch: 0
0 : 12.793858528137207
1 : 12.793899536132812
2 : 12.793862342834473
3 : 12.793856620788574

loss (epoch): 0.3998084142804146, change: 00.0%
validation loss: 0.39980801939964294
epoch: 1
0 : 12.793856620788574
1 : 12.793856620788574
2 : 12.793856620788574
3 : 12.793856620788574


KeyboardInterrupt: 

In [ ]:
model = torch.load('../evaluation/models/%s'%model_name)
print(model)
print(model_name)

In [ ]:
num_ex_test = 100
X_test_data, y_test_data = make_data(data['X_test'], data['y_test'], num_ex_test)

In [ ]:
dev_results = {}
seen_contexts = set()
seen_questions = set()
from hashlib import sha1

model.init_params(1)
for i, (x, y) in enumerate(zip(X_test_data, y_test_data)):
    qid = data['X_test'][i][0]
    if qid not in seen_questions:
        seen_questions.add(qid)
    else:
        continue
    c = data['X_test'][i][1]
    context_h = sha1(c.encode('utf-8')).hexdigest()
    if context_h not in seen_contexts:
        print(c)
        print("-"*30)
        seen_contexts.add(context_h)
    q = data['X_test'][i][2]
    print("QUESTION: ", q)
    a = data['X_test'][i][3]
    print("ANSWER: ", a)
    res = model.predict([x], bs=1).data.tolist()[0]
    if res[0]>res[1]:
        res[0], res[1] = res[1], res[0]
    ans = get_answer_span(res, c)
    print("PRED_SPAN: ", res)
    print("PREDICTED: ", ans)
    dev_results[qid] = ans
    print("="*50)
with open('../data/run_%sdata_%sepochs.json'%(num_ex, model.epochs), 'w') as f:
    json.dump(dev_results, f)